# Importing the Dependencies


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt 

In [ ]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [ ]:
# first 5 rows of the dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# dataset informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [ ]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()


0    284315
1       492
Name: Class, dtype: int64

This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [ ]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [ ]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [ ]:
# statistical measures of the data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [ ]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [ ]:
# compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under-Sampling


Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 492

In [ ]:
legit_sample = legit.sample(n=492)

Concatenating two DataFrames

In [ ]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [ ]:
new_dataset.reset_index(drop=True,inplace=True)
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,121941.0,-0.139993,1.033637,0.007097,-0.176276,1.308226,-0.020255,0.842686,0.003450,-0.853058,-0.717109,0.766939,0.532346,0.550911,-0.780471,-0.286011,0.127238,0.401370,0.551711,1.521372,0.153709,-0.234258,-0.632401,-0.094595,0.130496,-0.478490,0.315619,0.114193,0.174911,1.98,0
1,128556.0,1.574150,-0.707646,-2.168444,0.220266,0.732519,0.826606,-0.064385,0.329788,0.924123,-0.906353,1.159178,0.498577,-1.121382,-1.118719,0.119919,-0.400476,1.599711,-0.415115,-0.483789,0.028543,-0.074495,-0.294950,0.043883,-1.777349,-0.317925,0.009135,0.002703,-0.018673,174.91,0
2,72917.0,-2.727243,-1.952090,1.243363,1.539453,3.074675,-1.863299,-0.877170,0.069902,-0.236602,-0.020190,-1.511480,-0.369608,-0.336263,0.274344,0.526166,-0.564948,-0.055650,0.198004,1.193998,0.434619,0.091655,-0.140989,-0.071333,-0.442669,0.905633,-0.101792,-0.334534,0.631621,1.18,0
3,83209.0,-0.625657,1.126133,1.793135,0.223782,0.553442,-0.163470,0.842044,-0.053517,-0.768629,-0.636827,-1.027788,0.175860,1.134652,-0.042029,0.896677,0.359997,-0.804117,-0.007237,0.272833,0.092718,-0.257014,-0.800049,-0.238788,-0.491030,0.346406,-0.612759,0.069201,0.076354,14.07,0
4,78471.0,1.072014,0.010591,0.464287,1.351830,-0.322522,0.003897,-0.095548,0.188730,0.213423,0.073978,1.120777,0.589687,-1.557453,0.539156,-0.663354,-0.522154,0.162009,-0.384764,-0.151590,-0.233161,-0.040969,-0.023213,-0.049367,0.210002,0.555139,-0.340384,0.024383,0.008228,26.30,0


In [ ]:
new_dataset['Class'].value_counts()

1    492
0    492
Name: Class, dtype: int64

Splitting the data into Features & Targets


In [ ]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [ ]:
print(X)

         Time        V1        V2  ...       V27       V28  Amount
0    121941.0 -0.139993  1.033637  ...  0.114193  0.174911    1.98
1    128556.0  1.574150 -0.707646  ...  0.002703 -0.018673  174.91
2     72917.0 -2.727243 -1.952090  ... -0.334534  0.631621    1.18
3     83209.0 -0.625657  1.126133  ...  0.069201  0.076354   14.07
4     78471.0  1.072014  0.010591  ...  0.024383  0.008228   26.30
..        ...       ...       ...  ...       ...       ...     ...
979  169142.0 -1.927883  1.125653  ...  0.292680  0.147968  390.00
980  169347.0  1.378559  1.289381  ...  0.389152  0.186637    0.76
981  169351.0 -0.676143  1.126366  ...  0.385107  0.194361   77.89
982  169966.0 -3.113832  0.585864  ...  0.884876 -0.253700  245.00
983  170348.0  1.991976  0.158476  ...  0.002988 -0.015309   42.53

[984 rows x 30 columns]


In [ ]:
print(Y)

0      0
1      0
2      0
3      0
4      0
      ..
979    1
980    1
981    1
982    1
983    1
Name: Class, Length: 984, dtype: int64


Split the data into Training data & Testing Data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


## **Support Vector Classifier(SVC)**


Model Training

In [ ]:
model_svc = svm.SVC(kernel='linear')

In [ ]:
# training the Support Vector Classifier Model with Training Data
model_svc.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Model Evaluation

In [ ]:
# accuracy on training data
X_train_svc_prediction = model_svc.predict(X_train)
training_data_accuracy = accuracy_score(X_train_svc_prediction, Y_train)

In [ ]:
print('Accuracy score on Training Data : ', training_data_accuracy)

Accuracy score on Training Data :  0.8907242693773825


In [ ]:
# accuracy on test data
Y_prediction = model_svc.predict(X_test)
test_data_accuracy = accuracy_score(Y_prediction, Y_test)
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.883248730964467


### **Logistic** ***Regression***

Model Training

In [99]:
model_lr=LogisticRegression(random_state=1)

In [108]:
# training the Logistic Regression Model with Training Data
model_lr.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Model Evaluation

In [105]:
# accuracy on training data
X_train_lr_prediction = model_lr.predict(X_train)
training_data_accuracy = accuracy_score(X_train_lr_prediction, Y_train)

In [106]:
print('Accuracy score on Training Data : ', training_data_accuracy)


Accuracy score on Training Data :  0.9415501905972046


In [107]:
# accuracy on test data
Y_prediction = model_lr.predict(X_test)
test_data_accuracy = accuracy_score(Y_prediction, Y_test)
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.9289340101522843
